***<h1><b>Applied Machine Learning<b></h1>***
<h2>Αλέξανδρος - Ιωάννης Δουνάκης</h2> 
<h3>3170044</h3>

Εργασία 1 - 

In [11]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns

raw_data = pd.read_csv("2757408.csv",parse_dates=["DATE"] )
raw_data

,STATION,NAME,DATE,PRCP,TAVG,TMAX,TMIN
0,GR000016716,"HELLINIKON, GR",1955-01-01,0.00,NaN,65.0,50.0
1,GR000016716,"HELLINIKON, GR",1955-01-02,0.08,NaN,58.0,45.0
2,GR000016716,"HELLINIKON, GR",1955-01-03,0.00,NaN,60.0,49.0
3,GR000016716,"HELLINIKON, GR",1955-01-04,0.00,NaN,66.0,45.0
4,GR000016716,"HELLINIKON, GR",1955-01-05,0.00,NaN,64.0,47.0
...,...,...,...,...,...,...,...
23246,GR000016716,"HELLINIKON, GR",2020-12-27,0.00,62.0,NaN,NaN
23247,GR000016716,"HELLINIKON, GR",2020-12-28,0.26,59.0,65.0,53.0
23248,GR000016716,"HELLINIKON, GR",2020-12-29,0.00,61.0,68.0,54.0
23249,GR000016716,"HELLINIKON, GR",2020-12-30,0.00,62.0,68.0,59.0
